In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from scipy.stats import binom
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 40)

In [98]:
# Collecte des données

data=pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
zones = gpd.read_file('aires.geojson')

In [99]:
variables = ['PARC_2011','PARC_2012','PARC_2013','PARC_2014','PARC_2015','PARC_2016','PARC_2017','PARC_2018','PARC_2019','PARC_2020','PARC_2021','PARC_2022']
regions = data['REGION_LIBELLE'].unique()
regionsr = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Corse', 'Grand Est', 'Guadeloupe',
       'Guyane', 'Hauts-de-France', 'La Réunion',
       'Martinique', 'Mayotte', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmet = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmethorsb = ['Auvergne-Rhône-Alpes', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
critair = data['CRITAIR'].unique()

In [101]:
data.sample(5)

,REGION_CODE,REGION_LIBELLE,CLASSE_VEHICULE,CATEGORIE_VEHICULE,CARBURANT,AGE,STATUT_UTILISATEUR,CRITAIR,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
47522,11,Île-de-France,vp,Véhicule particulier,Diesel,24 ans,Particulier,Crit'Air 5,0,0,0,0,0,0,0,0,1,25,524,6186
15391,44,Grand Est,vul,Camionnette,Essence,12 ans,Professionnel,Non classé,107,77,55,32,21,10,3,1,2,0,0,0
19520,32,Hauts-de-France,vul,Camionnette,Gaz et inconnu,16 ans,Particulier,Crit'Air 1,0,0,0,5,32,39,29,19,31,33,26,30
3276,02,Martinique,tcp,Autocar,Diesel,9 ans,Particulier,Crit'Air 3,0,0,0,0,0,0,0,0,0,3,4,3
24480,28,Normandie,vul,Camionnette,Gaz et inconnu,19 ans,Professionnel,Crit'Air 1,0,0,0,0,0,0,1,10,13,8,3,4


In [102]:
data_fil = data.loc[(data["CATEGORIE_VEHICULE"] == "Véhicule particulier") | (data["CATEGORIE_VEHICULE"] == "Dérivé VP") | (data["CATEGORIE_VEHICULE"] == "Camionnette")]
pannes = np.ndarray((len(data_fil),))
data_fil.groupby(['REGION_CODE','CATEGORIE_VEHICULE','CARBURANT','AGE'])

for i in range (len(data_fil)) :
    pannes_annees = []
    for j in range (11, 22) :
        var = "PARC_20" + str(j)
        var_2 = "PARC_20" + str(j + 1)
        age = int(data_fil.iloc[i]["AGE"][0:2])
        if age == 0 :
            age_plus_1 = "1 an"
        elif age == 24 :
            age_plus_1 = "25 ans et plus"
        else : 
            age_plus_1 = str(age + 1) + " ans"
        data_plus_1 = data_fil.loc[
                        (data_fil["REGION_CODE"] == data_fil.iloc[i]["REGION_CODE"]) &
                        (data_fil["CATEGORIE_VEHICULE"] == data_fil.iloc[i]["CATEGORIE_VEHICULE"]) &
                        (data_fil["CARBURANT"] == data_fil.iloc[i]["CARBURANT"]) &
                        (data_fil["CRITAIR"] == data_fil.iloc[i]["CRITAIR"]) &
                        (data_fil["STATUT_UTILISATEUR"] == data_fil.iloc[i]["STATUT_UTILISATEUR"]) &
                        (data_fil["AGE"] == age_plus_1)
        ]
        if len(data_plus_1) > 0 :
            pannes_annees.append(data_fil.iloc[i][var] - data_plus_1.iloc[0][var_2])
    pannes[i] = np.mean(pannes_annees)

data_fil.insert(8, 'Pannes', pannes)


/opt/mamba/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/mamba/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [ ]:
data_fil.head(5)